# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import time

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kodiak,57.7900,-152.4072,12.73,76,100,2.57,US,1724968516
1,1,alofi,-19.0595,-169.9187,23.94,73,75,6.17,NU,1724968517
2,2,thompson,55.7435,-97.8558,17.09,94,75,1.54,CA,1724968518
3,3,port-aux-francais,-49.3500,70.2167,3.51,77,69,14.63,TF,1724968519
4,4,akashi,34.6333,134.9833,25.83,85,97,5.30,JP,1724968520


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    color="City",
    alpha=0.7,
    frame_width=800,
    frame_height=600,
    hover_cols=["City", "Country", "Humidity"]
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[
    (city_data_df["Max Temp"] > 21) & 
    (city_data_df["Max Temp"] < 27) & 
    (city_data_df["Wind Speed"] < 4.5) & 
    (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
131,131,weihai,37.5017,122.1136,21.28,81,0,2.20,CN,1724968666
148,148,zintan,31.9223,12.2520,24.55,58,0,1.59,LY,1724968685
157,157,puerto rico,-26.7960,-55.0240,22.15,64,0,1.20,AR,1724968504
229,229,gargalianoi,37.0667,21.6333,23.82,80,0,3.08,GR,1724968781
230,230,kerman,30.2832,57.0788,23.92,19,0,4.12,IR,1724968782


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
131,weihai,CN,37.5017,122.1136,81,
148,zintan,LY,31.9223,12.2520,58,
157,puerto rico,AR,-26.7960,-55.0240,64,
229,gargalianoi,GR,37.0667,21.6333,80,
230,kerman,IR,30.2832,57.0788,19,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",  # Corrected spelling from "catagories" to "categories"
    "apiKey": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        # Convert the API response to JSON format
        name_address = response.json()

        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        try:
            hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        except (KeyError, IndexError):
            # If no hotel is found, set the hotel name as "No hotel found".
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    else:
        print(f"Failed to retrieve data for {row['City']}. Status code: {response.status_code}")
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

    # Pause to avoid rate limiting
    time.sleep(1)

# Display sample data
hotel_df

Starting hotel search
weihai - nearest hotel: Huakun
zintan - nearest hotel: فندق الضيافة
puerto rico - nearest hotel: Suizo
gargalianoi - nearest hotel: Messinian Horizons
kerman - nearest hotel: حافظ
xindian - nearest hotel: 亚洲酒店
essaouira - nearest hotel: Essaouira Wind Palace
ibia - nearest hotel: Hotel Paulista
victoria point - nearest hotel: Seminara Appartments
al qubbah - nearest hotel: No hotel found
huilong - nearest hotel: 先豪国际酒店
al qusayr - nearest hotel: No hotel found
temiskaming shores - nearest hotel: Waterfront Inn


,City,Country,Lat,Lng,Humidity,Hotel Name
131,weihai,CN,37.5017,122.1136,81,Huakun
148,zintan,LY,31.9223,12.2520,58,فندق الضيافة
157,puerto rico,AR,-26.7960,-55.0240,64,Suizo
229,gargalianoi,GR,37.0667,21.6333,80,Messinian Horizons
230,kerman,IR,30.2832,57.0788,19,حافظ
236,xindian,CN,36.7975,118.2944,82,亚洲酒店
311,essaouira,MA,31.5125,-9.7700,78,Essaouira Wind Palace
492,ibia,BR,-19.4783,-46.5389,35,Hotel Paulista
503,victoria point,AU,-27.5833,153.3000,46,Seminara Appartments
518,al qubbah,LY,32.7616,22.2424,77,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    color="City",
    alpha=0.7,
    frame_width=800,
    frame_height=600,
    hover_cols=["City", "Country", "Hotel Name", "Humidity"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)